In [2]:
import pandas as pd
from sqlalchemy import create_engine, text


def execute_query(conn, query):
    """Executes a SQL query using the provided connection and returns the results as a list of tuples."""
    result = conn.execute(text(query))
    columns = result.keys()
    rows = result.fetchall()
    return rows, columns


# Database connection details (you provided these)
server = '10.10.11.241'
database = 'omar.rme1'
user = 'omar'
password = 'omar123'

# Create SQLAlchemy engine
connection_string = f'mssql+pyodbc://{user}:{password}@{server}/{database}?driver=SQL+Server'
engine = create_engine(connection_string)

try:
    with engine.connect() as conn:
        print("Connected to SQL Server successfully!")

        # Construct the SQL query with filters
        query = """
        SELECT *
        FROM mat_mov
        WHERE trx_type IN ('Move Order Issue on Project', 'RME Issue ( On Project)', 'RME Site Return')
        AND (item_desc LIKE '%خرسانة%' OR item_desc LIKE '%concrete%')  -- Either Arabic or English
        AND unit = 'CBM'
        """

        # Execute the query and fetch data
        results, columns = execute_query(conn, query)

        # Create a Pandas DataFrame
        df = pd.DataFrame(results)

        # Check if the DataFrame is empty
        if df.empty:
            print(
                "No concrete quantities found with the specified filters."
            )
        else:
            df.columns = columns
            # Excel file name
            excel_file_name = "concrete_quantities.xlsx"
            df.to_excel(excel_file_name, index=False)
            print(f"Data exported to {excel_file_name} successfully!")

except Exception as e:
    print("Error:", e)

Connected to SQL Server successfully!
Data exported to concrete_quantities.xlsx successfully!
